# 千帆 function call 入门

## 简介

function_call，顾名思义，通过给大模型提供 function 的说明描述，以及对应的入参出参 schema，让大模型输出 function 调用策略，结合多轮对话，以最终实现一个复杂的任务。
以下将以获取数据库中某类文件的数量为例子，通过调用千帆 Python SDK提供的 ERNIE 大模型以得到数据库中该语言的文件数量。

## 准备

本文使用了千帆 Python SDK中的 chat_completion 模块，该模块提供了与千帆对话引擎的交互接口，目前支持function call的模型有`ERNIE-Functions-8K`与`ERNIE-3.5-8K`。

首先安装千帆 Python SDK，版本号 >= 0.4.2。

In [8]:
%%capture
!pip install "qianfan" -U

初始化我们所需要的凭证

In [9]:
# 初始化LLM
import os

# qianfan sdk 鉴权
os.environ["QIANFAN_ACCESS_KEY"]="..."
os.environ["QIANFAN_SECRET_KEY"]="..."


自定义一个给chat调用的函数，此处以获取数据库中特定语言撰写的文件数量为例

In [1]:
def get_file_num(language: str) -> str:
    """获取数据库中指定语言的代码文件数量"""
    language_low = language.lower()
    language_map = {
        "c/c++": 35,
        "java": 10,
        "javascript": 25,
        "python": 35,
        "go": 32,
    }
    return str(language_map.get(language_low, 0))

描述函数调用策略
其中，name为函数名称，description为函数描述，properties为入参schema，required为必填参数
properties支持json schema格式，具体参考[json schema](https://json-schema.org/learn/getting-started-step-by-step.html


In [2]:
func_list = [{
    "name": "get_file_num",  # 函数名称
    "description": "获取内部数据库中以某一编程语言编写的文件数量",  # 函数描述
    "parameters":{
        "type":"object",
        "properties":{  # 参数schema，如果参数为空，设为空字典即可
            "language":{  # 参数名称
                "type":"string",  # 参数类型
                "description": "代码所运用的编程语言，例如：python、c/c++、go、java"  # 参数描述
            }
        },
        "required":["language"]  # 必填参数（无默认值）
    }
}]

创建 chat_completion 对象,调用 do 方法进行交互

In [3]:
import qianfan
import json

chat_comp = qianfan.ChatCompletion(model="ERNIE-Functions-8K")  # 指定模型
query = "请帮我查询一下数据库中用go撰写的代码文件数量"
msgs = qianfan.QfMessages()
msgs.append(query,role='user')
resp = chat_comp.do(
    messages=msgs,
    functions=func_list
)
print(resp['body']['result'])

[WARNING][2024-07-24 17:50:55.714] base.py:828 [t:8344509248]: This key `request_id` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept
[WARNING][2024-07-24 17:50:55.714] base.py:828 [t:8344509248]: This key `auto_concat_truncate` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept
[WARNING][2024-07-24 17:50:55.714] base.py:828 [t:8344509248]: This key `truncated_continue_prompt` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept
[WARNING][2024-07-24 17:50:55.715] base.py:828 [t:8344509248]: This key `truncate_overlong_msgs` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept
[INFO][2024-07-24 17:50:55.715] oauth.py:228 [t:8344509248]: trying to refresh access_token for ak `rRlk1M***`
[INFO][2024-07-24 17:50:55.863] oauth.py:243 [t:8344509248]: sucessfully refresh access_token


可以发现，此时chat的反馈为null，我们尝试打印返回值

In [4]:
print(resp.body)

{'id': 'as-9g8qscsdq5', 'object': 'chat.completion', 'created': 1721814657, 'result': '', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 226, 'completion_tokens': 17, 'total_tokens': 243}, 'function_call': {'name': 'get_file_num', 'arguments': '{"language": "go"}'}}


从thoughts可见chat确定需要调用函数get_file_num，我们需要将反馈作为function输入来进行二次对话

In [5]:
if resp.get("function_call"):
    # 获取函数名称、入参及返回值
    func_call_result = resp["function_call"]
    func_name = func_call_result["name"]
    language = json.loads(func_call_result["arguments"]).get("language")
    func_resp = get_file_num(language)
    
    # 将函数返回值转换成json字符串
    func_content = json.dumps({
        "return":func_resp
    })
    
    # 创建新的消息
    msgs.append(resp, role="assistant")
    msgs.append(func_content, role="function")
    
    # 再次调用chat_completion
    second_resp = chat_comp.do(
        messages=msgs,
        functions=func_list
    )
    
    print(second_resp['body']['result'])

[WARNING][2024-07-24 17:51:00.753] base.py:828 [t:8344509248]: This key `request_id` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept
[WARNING][2024-07-24 17:51:00.754] base.py:828 [t:8344509248]: This key `auto_concat_truncate` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept
[WARNING][2024-07-24 17:51:00.754] base.py:828 [t:8344509248]: This key `truncated_continue_prompt` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept
[WARNING][2024-07-24 17:51:00.755] base.py:828 [t:8344509248]: This key `truncate_overlong_msgs` does not seem to be a parameter that the model `ERNIE-Functions-8K` will accept


数据库中用Go撰写的代码文件数量为32。


通过以上步骤，我们成功让chat调用了函数get_file_num，并得到了正确的返回值。
接下来，我们可以尝试让chat调用千帆自带的工具函数
[千帆function_call工具调用](./function_call_with_tool.ipynb)